<a href="https://colab.research.google.com/github/sbowma15/gpt2_text_generation/blob/main/gpt2_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -m install pandas matplotlib numpy/
nltk seaborn sklearn gensim pyldavis
wordcloud textblob spacy textstat


import torch
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cpierse/gpt2_film_scripts")
model = AutoModelWithLMHead.from_pretrained("cpierse/gpt2_film_scripts")

# Function to first select topN tokens from the probability list and then based on the N word distribution
# select random token ID
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

model.eval()
with torch.no_grad():

    for i in range(250):
        outputs = model(cur_ids, labels=cur_ids)
        loss, logits = outputs[:2]
        softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(only one) batch and the last predicted embedding
        next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=5) #Randomly(from the given probability distribution) choose the next word from the top n words
        cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word

    output_list = list(cur_ids.squeeze().to('cpu').numpy())
    output_text = tokenizer.decode(output_list)
#print(output_text)

decoded_output = []
for sample in output_list:
        decoded_output.append(tokenizer.decode(
            sample, skip_special_tokens=True))
print(decoded_output[0])

SyntaxError: ignored